In [1]:
import pandas as pd
import numpy as np
import json, os
import matplotlib.pylab as plt
from SBRP_DI import visualization, aux, fileOperations as fo
from SBRP_DI.caseStudies import postprocessing as pp

## loading data

In [2]:
# clustering
clustering = pp.loadData('hull_pm_general', res_type='clustering')
clustering_gb_max = clustering.groupby('time').max()

clustering_all = pp.loadData('hull_pm_general', res_type='clustering', path_root='../run2/times/')
clustering_all_gb_max = clustering_all.groupby('time').max()

# routing
sch17 = pp.loadData('sch17')
hull = pp.loadData('hull_pm_general')
sch17_all = pp.loadData('sch17', path_root='../run2/times/')
hull_all = pp.loadData('hull_pm_general', path_root='../run2/times/')


sch17_gb_max = sch17.groupby('time').max()
sch17_all_gb_max = sch17_all.groupby('time').max()
hull_gb_max = hull.groupby('time').max()
hull_all_gb_max = hull_all.groupby('time').max()

mask = sch17_gb_max['ObjVal'] != np.inf
mask_all = sch17_all_gb_max['ObjVal'] != np.inf
m = mask & mask_all

In [3]:
mask[~mask]

time
2021-06-01    False
2021-06-02    False
2021-06-03    False
2021-06-08    False
2021-06-10    False
2021-06-11    False
2021-06-14    False
2021-06-15    False
2021-06-17    False
2021-06-22    False
2021-06-23    False
2021-07-05    False
2021-07-12    False
Name: ObjVal, dtype: bool

In [4]:
m[~m]

time
2021-06-01    False
2021-06-02    False
2021-06-03    False
2021-06-07    False
2021-06-08    False
2021-06-10    False
2021-06-11    False
2021-06-14    False
2021-06-15    False
2021-06-17    False
2021-06-21    False
2021-06-22    False
2021-06-23    False
2021-06-28    False
2021-07-02    False
2021-07-05    False
2021-07-12    False
Name: ObjVal, dtype: bool

In [5]:
(
pp.count_no_solution(sch17_gb_max),
pp.count_no_solution(sch17_all_gb_max),
pp.count_no_solution(hull_gb_max),
pp.count_no_solution(hull_all_gb_max)
)

(13, 14, 0, 0)

In [6]:
S_S0_array = pp.load_S_S0(root='times/')

## results where we have an integer result with sch17 

In [7]:
table = pp.initTable(family='region C / 3-6 pm', S=150, S_S0=round(S_S0_array.mean(), 2))
table.loc[0, 'picking strategy'] = 'subset'
table.loc[0, 'clust_gap']  = round(clustering_gb_max[mask]['MIPGap'].mean(), 2) * 100
table.loc[0, 'clust_time'] = round(clustering_gb_max[mask]['Runtime'].mean(), 1)
pp.rowToTable(table, [sch17_gb_max[mask], hull_gb_max[mask]],
                        ['sch17', 'hull'], row=0, latex=False)

table.loc[1, 'picking strategy'] = 'all'
table.loc[1, 'clust_gap']  = round(clustering_all_gb_max[mask_all]['MIPGap'].mean(), 2) * 100
table.loc[1, 'clust_time'] = round(clustering_all_gb_max[mask_all]['Runtime'].mean(), 1)
pp.rowToTable(table, [sch17_all_gb_max[mask_all], hull_all_gb_max[mask_all]],
                        ['sch17', 'hull'], row=1, latex=True)

\begin{tabular}{llllrrrrrrr}
\toprule
            family &    S &  S\_S0 & picking strategy &  clust\_gap &  clust\_time &  sch17\_gap &  sch17\_time &  hull\_gap &  hull\_time &  hull\_rel\_diff \\
\midrule
 region C / 3-6 pm &  150 &  91.4 &           subset &        1.0 &       669.5 &       8.38 &      6530.7 &      0.08 &      637.1 &          -3.69 \\
               NaN &  NaN &   NaN &              all &        6.0 &      1348.7 &       8.10 &      5851.5 &      0.07 &      469.7 &          -3.65 \\
\bottomrule
\end{tabular}



## all results

In [8]:
table = pp.initTable(family='region C / 3-6 pm', S=150, S_S0=round(S_S0_array.mean(), 2))
table.loc[0, 'picking strategy'] = 'subset'
table.loc[0, 'clust_gap'] = round(clustering_gb_max['MIPGap'].mean(), 2) * 100
table.loc[0, 'clust_time'] = round(clustering_gb_max['Runtime'].mean(), 1)
pp.rowToTable(table, [hull_gb_max],
                     ['hull'], row=0, latex=False)

table.loc[1, 'picking strategy'] = 'all'
table.loc[1, 'clust_gap'] = round(clustering_all_gb_max['MIPGap'].mean(), 2) * 100
table.loc[1, 'clust_time'] = round(clustering_all_gb_max['Runtime'].mean(), 1)
pp.rowToTable(table, [hull_all_gb_max],
                     ['hull'], row=1, latex=True)

\begin{tabular}{llllrrrr}
\toprule
            family &    S &  S\_S0 & picking strategy &  clust\_gap &  clust\_time &  hull\_gap &  hull\_time \\
\midrule
 region C / 3-6 pm &  150 &  91.4 &           subset &        1.0 &       745.4 &      0.19 &     1669.8 \\
               NaN &  NaN &   NaN &              all &        8.0 &      1390.6 &      0.09 &      971.8 \\
\bottomrule
\end{tabular}



## differences

In [9]:
pp.difference(sch17_gb_max[m], sch17_all_gb_max[m]).mean()

0.6929039379379172

In [10]:
diffs_hull = pp.difference(hull_gb_max, hull_all_gb_max)
diffs_hull.min(), diffs_hull.mean(), diffs_hull.max()

(-7.000530519350455, -0.5358696235271083, 6.72712805152726)